# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
# Import API key
from api_keys import geoapify_key

C:\Users\d_ang\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,arraial do cabo,-22.9661,-42.0278,20.92,83,100,5.14,BR,1724615579
1,1,papatowai,-46.5619,169.4708,3.61,88,82,2.31,NZ,1724615581
2,2,miracema do tocantins,-9.5618,-48.3967,37.50,16,0,2.11,BR,1724615582
3,3,ushuaia,-54.8000,-68.3000,8.81,61,40,5.66,AR,1724615583
4,4,iqaluit,63.7506,-68.5145,7.85,75,100,1.54,CA,1724615584


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

city_map = city_data_df.hvplot.points(
        'Lng', 'Lat', geo=True, tiles='OSM', 
        size='Humidity', color='Humidity', 
        hover_cols=['City', 'Country', 'Humidity', 'Max Temp'], alpha=0.6, 
        frame_width=800, frame_height=600, title='Humidity in Cities Map', colorbar=True, 
        cmap='coolwarm', xlabel='Longitude', ylabel='Latitude')

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

#filtered_cities = city_data_df[(city_data_df['Humidity'] > 50) & (city_data_df['Lat'] > 20)]
filtered_cities = city_data_df[(city_data_df['Max Temp'] < 27) & 
                               (city_data_df['Max Temp'] > 21) & 
                               (city_data_df['Wind Speed'] < 4.5) & 
                               (city_data_df['Cloudiness'] == 0)]
# Drop any rows with null values
filtered_cities = filtered_cities.dropna()


# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,al hamul,31.3115,31.1477,25.26,76,0,1.95,EG,1724615599
41,41,jamestown,42.0970,-79.2353,26.80,41,0,2.06,US,1724615631
48,48,maun,-19.9833,23.4167,23.98,19,0,4.12,BW,1724615639
131,131,urumqi,43.8010,87.6005,21.53,53,0,2.00,CN,1724615748
172,172,kandahar,31.0000,65.7500,25.44,9,0,2.45,AF,1724615802
195,195,gaigeturi,33.4644,126.3183,26.03,73,0,2.57,KR,1724615830
256,256,andrushivka,50.0229,29.0202,23.00,35,0,3.43,UA,1724615906
269,269,newman,37.3138,-121.0208,26.70,43,0,2.68,US,1724615923
334,334,fortuna,40.5982,-124.1573,21.31,69,0,4.12,US,1724616006
372,372,hasaki,35.7333,140.8333,26.56,85,0,4.34,JP,1724616056


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
16,al hamul,EG,31.3115,31.1477,76,
41,jamestown,US,42.0970,-79.2353,41,
48,maun,BW,-19.9833,23.4167,19,
131,urumqi,CN,43.8010,87.6005,53,
172,kandahar,AF,31.0000,65.7500,9,
195,gaigeturi,KR,33.4644,126.3183,73,
256,andrushivka,UA,50.0229,29.0202,35,
269,newman,US,37.3138,-121.0208,43,
334,fortuna,US,40.5982,-124.1573,69,
372,hasaki,JP,35.7333,140.8333,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel", "apiKey": geoapify_key, "limit": 10}



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
al hamul - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
maun - nearest hotel: Center Lodge Conference Center
urumqi - nearest hotel: 百时快捷酒店红山店
kandahar - nearest hotel: No hotel found
gaigeturi - nearest hotel: 제주 토비스콘도
andrushivka - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
hasaki - nearest hotel: Choshi Plaza Hotel
ghormach - nearest hotel: No hotel found
kuqa - nearest hotel: Kuqa Hotel - foreigners accepted
qarah zia' od din - nearest hotel: No hotel found
mil'kovo - nearest hotel: Дрозды


,City,Country,Lat,Lng,Humidity,Hotel Name
16,al hamul,EG,31.3115,31.1477,76,No hotel found
41,jamestown,US,42.0970,-79.2353,41,DoubleTree Jamestown
48,maun,BW,-19.9833,23.4167,19,Center Lodge Conference Center
131,urumqi,CN,43.8010,87.6005,53,百时快捷酒店红山店
172,kandahar,AF,31.0000,65.7500,9,No hotel found
195,gaigeturi,KR,33.4644,126.3183,73,제주 토비스콘도
256,andrushivka,UA,50.0229,29.0202,35,No hotel found
269,newman,US,37.3138,-121.0208,43,No hotel found
334,fortuna,US,40.5982,-124.1573,69,Comfort Inn & Suites Redwood Country
372,hasaki,JP,35.7333,140.8333,85,Choshi Plaza Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
city_map = hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, tiles='OSM',
    size='Humidity', color='Humidity',
    hover_cols=['City', 'Country', 'Humidity', 'Max Temp', 'Hotel Name'],
    alpha=0.6, frame_width=800, frame_height=600, title='Humidity in Cities with Hotels',
    colorbar=True, cmap='coolwarm', xlabel='Longitude', ylabel='Latitude')

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)